In [25]:
import math
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


In [2]:
df = pd.read_csv('spotify_data.csv')

In [33]:
df.shape

(595, 16)

In [3]:
df['Release Date'] = pd.to_datetime(df['Release Date'])
df = df.sort_values(by='Release Date')


In [4]:
fig = px.scatter(df, x='Release Date', y='Tempo', color='Key', hover_name='Track Name', 
                 title='Time Series of Tempo and Key based on Release Date', 
                 labels={'Tempo': 'Tempo (BPM)', 'Release Date': 'Release Date', 'Key': 'Key'})

# Customize the layout
fig.update_traces(marker=dict(size=8, opacity=0.7),
                  selector=dict(mode='markers'))

# Show the plot
fig.show()

In [5]:
df2 = df[df['Release Date'].dt.year < 1975]

In [6]:
fig = px.scatter(df2, x='Release Date', y='Tempo', color='Key', hover_name='Track Name', 
                 title='Time Series of Tempo and Key based on Release Date', 
                 labels={'Tempo': 'Tempo (BPM)', 'Release Date': 'Release Date', 'Key': 'Key'})

# Customize the layout
fig.update_traces(marker=dict(size=8, opacity=0.7),
                  selector=dict(mode='markers'))

# Show the plot
fig.show()

In [7]:
df2.head()

,Track Name,Album Name,Tempo,Key,Mode,Release Date,Energy,Danceability,Loudness,Speechiness,Acousticness,Instrumentalness,Liveness,Time Signature,Valence,Duration
594,Getting Better,Sgt. Pepper's Lonely Hearts Club Band (Super D...,121.904,0,1,1967-05-26,0.444,0.590,-11.472,0.0394,0.1310,0.000051,0.1300,4,0.718,167253
566,Penny Lane - Stereo Mix 2017,Sgt. Pepper's Lonely Hearts Club Band (Super D...,112.921,9,1,1967-05-26,0.577,0.617,-5.710,0.0388,0.0644,0.001570,0.1060,4,0.627,180693
565,Penny Lane - Vocal Overdubs And Speech,Sgt. Pepper's Lonely Hearts Club Band (Super D...,113.267,7,1,1967-05-26,0.143,0.879,-19.917,0.2610,0.7000,0.001840,0.4520,4,0.393,107093
564,Penny Lane - Take 6 / Instrumental,Sgt. Pepper's Lonely Hearts Club Band (Super D...,120.775,0,1,1967-05-26,0.548,0.901,-6.808,0.0996,0.6670,0.837000,0.1610,3,0.135,175547
563,When I'm Sixty-Four - Take 2,Sgt. Pepper's Lonely Hearts Club Band (Super D...,132.619,7,1,1967-05-26,0.218,0.816,-12.286,0.1590,0.3200,0.000000,0.0846,4,0.434,181160


In [8]:
df2.columns

Index(['Track Name', 'Album Name', 'Tempo', 'Key', 'Mode', 'Release Date',
       'Energy', 'Danceability', 'Loudness', 'Speechiness', 'Acousticness',
       'Instrumentalness', 'Liveness', 'Time Signature', 'Valence',
       'Duration'],
      dtype='object')

In [31]:
df2.shape

(177, 16)

In [23]:
# Define the list of audio features
audio_features = ['Tempo', 'Key', 'Energy', 'Danceability', 'Loudness', 
                  'Speechiness', 'Acousticness', 'Instrumentalness', 
                  'Liveness', 'Time Signature', 'Valence', 'Duration']



In [37]:
# Define the height of the layout
layout_height = 350 * (len(audio_features) // 2 + 1)  # Increase factor (e.g., 400) to adjust height

# Create subplots with specified rows and columns
fig = make_subplots(rows=len(audio_features)//2 + 1, cols=2,
                    subplot_titles=audio_features, 
                    vertical_spacing=0.02)  # Adjust vertical spacing as needed

# Get Plotly colors
colors = px.colors.qualitative.Plotly

# Iterate over each audio feature
for i, feature in enumerate(audio_features):
    # Add a histogram for the current audio feature to the subplot
    row = i // 2 + 1
    col = i % 2 + 1
    fig.add_trace(go.Histogram(x=df2[feature], name=feature, marker_color=colors[i % len(colors)]), row=row, col=col)

# Update axes and layout
fig.update_xaxes(title_text='Value', row=len(audio_features)//2 + 1, col=1)
fig.update_yaxes(title_text='Count', row=len(audio_features)//2 + 1, col=1)
fig.update_layout(barmode='overlay', height=layout_height)

# Show the plot
fig.show()

In [42]:
# Normalize Loudness values
# max_loudness = df2['Loudness'].max()
# df2['Normalized Loudness'] = df2['Loudness'] / max_loudness

# Create Bubble Chart
fig = go.Figure()

# Add traces for each track
for index, row in df2.iterrows():
    fig.add_trace(go.Scatter(
        x=[row['Tempo']],
        y=[row['Duration']],
        mode='markers',
        marker=dict(
            size=row['Energy'] * 10,  # Adjust the size of the bubbles based on Energy
            color=row['Valence'],  # Use Valence for the color of the bubbles
            colorscale='RdBu',  # Set the color scale from red to blue
            cmin=df2['Valence'].min(),  # Set the minimum value for the color scale
            cmax=df2['Valence'].max(),  # Set the maximum value for the color scale
            opacity= 0.7, # row['Normalized Loudness'],  # Set the opacity based on Normalized Loudness
            colorbar=dict(
                title='Valence'  # Label for the colorbar
            )
        ),
        text=df2['Track Name'],  # Text to display when hovering over the bubbles
        hoverinfo='text'
    ))

# Update layout
fig.update_layout(
    title='Bubble Chart of Tempo vs Duration',
    xaxis=dict(title='Tempo'),
    yaxis=dict(title='Duration')
)

# Show the plot
fig.show()

In [47]:
# Create Bubble Chart
fig = go.Figure()

# Add traces for each track
for index, row in df2.iterrows():
    fig.add_trace(go.Scatter(
        x=[row['Energy']],
        y=[row['Duration']],
        mode='markers',
        marker=dict(
            size=abs(row['Loudness']) * 2,  # Take the absolute value of Loudness and adjust the size of the bubbles
            color=row['Valence'],  # Use Valence for the color of the bubbles
            colorscale='RdYlBu',  # Set the color scale from red to blue
            cmin=df2['Valence'].min(),  # Set the minimum value for the color scale
            cmax=df2['Valence'].max(),  # Set the maximum value for the color scale
            opacity=0.7,
            colorbar=dict(
                title='Valence'  # Label for the colorbar
            )
        ),
        text=row['Track Name'],  # Text to display when hovering over the bubbles
        hoverinfo='text'
    ))

# Update layout
fig.update_layout(
    title='Bubble Chart of Energy vs Duration',
    xaxis=dict(title='Energy'),
    yaxis=dict(title='Duration')
)

# Show the plot
fig.show()

In [55]:
# Create Bubble Chart
fig = go.Figure()

# Add traces for each track
for index, row in df2.iterrows():
    fig.add_trace(go.Scatter(
        x=[row['Tempo']],
        y=[row['Duration']],
        mode='markers',
        marker=dict(
            size= row['Energy']*50,  # Take the absolute value of Loudness and adjust the size of the bubbles
            color=row['Valence'],  # Use Valence for the color of the bubbles
            colorscale='RdBu',  # Set the color scale from red to blue
            cmin=df2['Valence'].min(),  # Set the minimum value for the color scale
            cmax=df2['Valence'].max(),  # Set the maximum value for the color scale
            opacity=0.7,
            colorbar=dict(
                title='Valence'  # Label for the colorbar
            )
        ),
        text=row['Track Name'],  # Text to display when hovering over the bubbles
        hoverinfo='text'
    ))

# Update layout
fig.update_layout(
    height=800,
    title='Bubble Chart of Energy vs Duration',
    xaxis=dict(title='Energy'),
    yaxis=dict(title='Duration')
)

# Show the plot
fig.show()

In [58]:
# Create Bubble Chart
fig = go.Figure()

# Add traces for each track
for index, row in df2.iterrows():
    fig.add_trace(go.Scatter(
        x=[row['Tempo']],
        y=[row['Duration']],
        mode='markers',
        marker=dict(
            size= row['Energy']*50,  # Take the absolute value of Loudness and adjust the size of the bubbles
            color=row['Valence'],  # Use Valence for the color of the bubbles
            colorscale='Inferno',  # Set the color scale from red to blue
            cmin=df2['Valence'].min(),  # Set the minimum value for the color scale
            cmax=df2['Valence'].max(),  # Set the maximum value for the color scale
            opacity=0.7,
            colorbar=dict(
                title='Valence'  # Label for the colorbar
            )
        ),
        text=row['Track Name'],  # Text to display when hovering over the bubbles
        hoverinfo='text'
    ))

# Update layout
fig.update_layout(
    height=800,
    title='Bubble Chart of Energy vs Duration',
    xaxis=dict(title='Energy'),
    yaxis=dict(title='Duration')
)

# Show the plot
fig.show()

In [ ]:
test